In [1]:
%pylab inline
import pysal as ps
import numpy as np
import util



Populating the interactive namespace from numpy and matplotlib


In [2]:
np.inf > 1000

True

In [15]:
ntw = ps.Network(ps.examples.get_path('geodanet/streets.shp'))

#Snap a point pattern to the network
ntw.snapobservations(ps.examples.get_path('geodanet/crimes.shp'), 'crimes', attribute=True)
ntw.snapobservations(ps.examples.get_path('geodanet/schools.shp'), 'schools', attribute=False)

In [16]:
import time
t1 = time.time()
ntw.snapobservations(ps.examples.get_path('geodanet/crimes.shp'), 'crimes', attribute=True)
t2 = time.time()
treeTime = t2-t1


In [17]:
pp = ntw.pointpatterns['crimes']

In [18]:
def snapbf(patternFile, network):
    pf = ps.open(ps.examples.get_path(patternFile))
    points = {}
    for i,p in enumerate(pf):
        points[i] = p
        
    nPoints = len(points)
    edges = {}
    for edge in ntw.edges:
        head, tail = edge
        hc = ntw.node_coords[head]
        tc = ntw.node_coords[tail]
        edges[edge] = [hc,tc]
    
    nEdges = len(edges)
    p2s = {}
    for pIdx,point in points.iteritems():
        dmin = np.inf
        for eIdx, edge in edges.iteritems():
            dpe = util.squaredDistancePointSegment(point, edge)
            if dpe[0] < dmin:
                p2s[pIdx] = [eIdx, dpe]
                dmin = dpe[0]
    
    return points, p2s
    

In [19]:
t1 = time.time()
res = snapbf('geodanet/crimes.shp', ntw)
t2 = time.time()
bfTime = t2-t1

print "Speedup: ", bfTime/treeTime
print "bf: ", bfTime
print "tree: ", treeTime

Speedup:  5.82892918005
bf:  2.00368189812
tree:  0.343747854233


In [3]:
ntw = ps.Network(ps.examples.get_path('newHaven/newhaven_nework.shp'))

In [5]:
%prun ntw.snapobservations(ps.examples.get_path('newHaven/new_haven_merged.shp'), 'dataset', attribute=True)

In [22]:
t1 = time.time()
ntw.snapobservations(ps.examples.get_path('newHaven/new_haven_merged.shp'), 'dataset', attribute=True)
t2 = time.time()
treeTime = t2-t1
print treeTime

161.934916973


In [23]:
t1 = time.time()
res = snapbf('newHaven/new_haven_merged.shp', ntw)
t2 = time.time()
bfTime = t2-t1

print "Speedup: ", bfTime/treeTime
print "bf: ", bfTime
print "tree: ", treeTime

Speedup:  6.53942316941
bf:  1058.96094799
tree:  161.934916973
